# Programming Machine Learning Lab
# Exercise 08

**General Instructions:**

1. You need to submit the PDF as well as the filled notebook file.
1. Name your submissions by prefixing your matriculation number to the filename. Example, if your MR is 12345 then rename the files as **"12345_Exercise_08.xxx"**
1. Complete all your tasks and then do a clean run before generating the final PDF. (_Clear All Ouputs_ and _Run All_ commands in Jupyter notebook)

**Exercise Specific instructions::**

1. You are allowed to use only NumPy and Pandas (unless stated otherwise). You can use any library for visualizations.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import nltk
from nltk.corpus import stopwords
import os
import scipy.sparse as sprs
from scipy.sparse import lil_matrix
from collections import Counter
from nltk.corpus import stopwords

### Part 1

**TF-IDF and BOW**

In this part, you will be working with the IMBD movie review dataset to perform various natural language processing tasks. You need to get the dataset from https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

1. Download and read the dataset (subset the data to only use 10,000 rows).
1. Perform tokenization on the review text.
1. Remove stop words from the tokenized text.
1. Use regular expressions to clean the text, removing any HTML tags, emails, and other unnecessary information.
1. Convert the cleaned data into a TF-IDF and BOW representation from scratch.

*Note: you can use NLTK for all sub-parts except the last*

**Main task**:
Using the BOW and Tf-Idf representation, implement a Naive-Bayes classifier for the data from scratch. Use Laplace smoothing for the implementation **Do not use sklearn for this part** 

[Reference Slide](https://www.ismll.uni-hildesheim.de/lehre/ml-16w/script/ml-09-A8-bayesian-networks.pdf)

In [3]:
### Your code here

def preprocess_txt(text):

    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)

    # convert text to lowercase
    text = text.strip().lower()

    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    stop_words = set(stopwords.words('english')) # setting stopwords of English language
    words = text.split() # create a list of words
    filtered = ' '.join([word for word in words if not word in stop_words]) # removing stopwords
    return filtered

#cleaning the reviews from html tags, urls, stopwords and punctutation
df = pd.read_csv("IMDB Dataset.csv")
df['review'] = df['review'].apply(preprocess_txt)

In [5]:
class NLPModels:
    def __init__(self, col):
        self.col = col #column in which text data is stored
        
    def fit(self):
        self.count_words_across_document = {} #this is used to track the number of comments which contain a particular word
        self.list_words_sentences = [] #this list tracks word counts of each comment
        for sent in self.col: #iterating over each comment
            count_in_sentence = {} #this tracks the word seen in comment so far and its count
            for word in sent.split(): #iterating over each word in a comment
                
                #if word is already in dictionary increase the count else make a new key
                if word in self.count_words_across_document:
                    self.count_words_across_document[word]+=1
                    
                elif word not in self.count_words_across_document:
                    self.count_words_across_document[word]=1
                
                #this tracks word in each comment only
                if word in count_in_sentence:
                    count_in_sentence[word]+=1

                elif word not in count_in_sentence:
                    count_in_sentence[word]=1
            
            #make a list containing all the words across all comments
            self.list_words_sentences.append(count_in_sentence)
            
        #to make a two way connection between going from words to index or vice versa
        self.idx_to_word= {i:w for i,w in enumerate(self.count_words_across_document)}
        self.word_to_idx = {w:i for i,w in enumerate(self.count_words_across_document)}
        self.vocab_size = len(self.count_words_across_document) #number of unique words
            
    def BOW(self):
        bow = sprs.lil_matrix((1,self.vocab_size)) #sparse matrix as they are more efficent
        for sent in self.list_words_sentences: #iterate over each comment words dictionary
            rep = sprs.lil_matrix((1,self.vocab_size)) #empty matrix to store the final representation of the word
            for word, count in sent.items(): #iterating ober each word
                rep[0,self.word_to_idx[word]]=count #replacing the count in the representation matrix of that word
            bow = sprs.vstack([bow,rep]) #concatenate the matrix vertically to obtain one big matrix of size mxd
        return bow.toarray()[1:,:] 
    
    def TF_IDF(self):
        tokenized_docs,vocab,word_index=self.vocab_index(self.col)
        word_counts = Counter([word for doc in tokenized_docs for word in set(doc)])

        tf_matrix=lil_matrix((len(tokenized_docs),len(vocab)),dtype=np.float64)

        for i,doc in enumerate(tokenized_docs):
            for word in doc:
                tf_matrix[i,word_index[word]]=doc.count(word)

        idf_matrix = lil_matrix((len(vocab),len(vocab)),dtype=np.float64)

        for i,word in enumerate(vocab):
            idf_matrix[i,i]=np.log((len(tokenized_docs)+1)/word_counts[word])

        tf_idf_matrix=tf_matrix*idf_matrix
        return tf_idf_matrix.toarray()

    def vocab_index(self,X):
        tokenized_docs=[preprocess_txt(X.iloc[i]).split() for i in range(self.col.shape[0])]
        vocab=set([word for doc in tokenized_docs for word in doc])
        word_index = {word :i for i,word in enumerate(vocab)}
        return tokenized_docs,vocab,word_index

In [6]:
#let's make the model using only the first 1000 reviews, the code will remain same for the analysis over whole data
n = NLPModels(df['review'].iloc[:1000]) 
#fit the model
n.fit()
#obtain the BOW representation
bow_representation = n.BOW()
#obtain TF-IDF representation
tfidf_representation = n.TF_IDF()

In [7]:
y = np.where(df.iloc[:1000,-1]=='positive',1,0) #to make reviews binary where 1=positive and 0=negative review
#first using the bow representation
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow_representation,y,random_state=42, test_size=0.2)
#let's try with TF-IDF representation now
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_representation,y,random_state=42, test_size=0.2)

In [8]:
class NaiveBayes:
    def __init__ (self, alpha, vocab_size):
        self.alpha = alpha #smoothing parameter alpha
        self.vocab_size = vocab_size #vocab size of the words
        
    def fit(self, X, y):
        self.X = X #store the features
        self.y = y #store the targets
        m,n = self.X.shape
        
        #num classes and counts of unique classes in data
        self.classes, counts = np.unique(self.y,return_counts=True)
        n_classes = len(self.classes)
        
        #To store conditional probability of all the words
        self.cond_prob = np.zeros(shape=(n,n_classes))
        
        #To store prior probability of all the words 
        self.prior_prob = [cnt/m for cnt in counts]
        
        #iterate over each class
        for cls in self.classes:
            idx = self.y==cls #indices of all instances which belong to particular class
            sub_x = self.X[idx,:] #sample the dataset of all observation belonging to particular class
            
            #calculate the conditional probability using laplacian smoothing
            self.cond_prob[:,cls] = (sub_x.sum(axis=0) + self.alpha)/(sub_x.sum() + self.alpha * self.vocab_size)
        
    def predict(self,X):
        preds = [] #to store the predictions
        for x in X: #iterative over all test instances
            class_prob = [] #store probability for each instance
            for cls in self.classes: #iterate over all classes
                idx = self.y == cls #store indices of particular class samples
                sub_x = self.X[idx,:] #sample the observation belonging to particular class
                
                #to calculate the probability
                likelihood = np.multiply(np.log(self.cond_prob[:,cls].reshape(1,-1)),x.reshape(1,-1))
                prob = np.sum(likelihood) + np.log(self.prior_prob[cls])
                
                #store the probability of belonging to each class
                class_prob.append(prob)
            
            #the class with maximum probability is the class of an instance
            preds.append(np.argmax(class_prob))
        return preds

In [9]:
N = NaiveBayes(alpha=1,vocab_size=n.vocab_size)
N.fit(X_train_bow,y_train_bow)
print('The Accuracy of the model while using BOW on test set is',accuracy_score(y_test_bow,N.predict(X_test_bow)))

N = NaiveBayes(alpha=1,vocab_size=n.vocab_size)
N.fit(X_train_tfidf,y_train_tfidf)
print('The Accuracy of the model while using TF-IDF on test set is',accuracy_score(y_test_tfidf,N.predict(X_test_tfidf)))

The Accuracy of the model while using BOW on test set is 0.795
The Accuracy of the model while using TF-IDF on test set is 0.735


**Evaluation**

Use sklearn implementation of Naive-Bayes classifier and compare the results with your implementation.

In [10]:
### Your code here
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_bow,y_train_bow)
print('The Accuracy of the Sklearn model while using BOW on test set is',accuracy_score(y_test_bow,nb.predict(X_test_bow)))

#to compare from sklearn's implementation
nb = MultinomialNB()
nb.fit(X_train_tfidf,y_train_tfidf)
print('The Accuracy of the Sklearn model while using TF-IDF on test set is',accuracy_score(y_test_tfidf,nb.predict(X_test_tfidf)))

The Accuracy of the Sklearn model while using BOW on test set is 0.795
The Accuracy of the Sklearn model while using TF-IDF on test set is 0.735


We can see that the Accuracy of our model using both BOW and TFIDF and the accuracy of Sklearn's implementation are same

### Part 2

**N-gram Language Model**


You won't believe what happened ??? !

Is the word "next" on the tip of your tongue? Although there are other possibilities, that is undoubtedly the most likely one. Other options are "after", "after that", and "to them". Our intuition tells us that some sentence endings are more plausible than others, especially when we take into account the previous information, the location of the phrase, and the speaker or author.

N-gram language models simply formalize that intuition. An n-gram model gives each possibility a probability score by solely taking into account the words that came before it. The probability of the word "next" in our example may be 80\%, whereas the probabilities of the words "after" and "then" might be 10\%, 5\%, and 5\%, respectively.

By leveraging these statistics, n-grams fuel the development of language models, which in turn contribute to an overall speech recognition system.

**Main task**:

In this part you are tasked with coding a N-gram language model on the dataset (https://www.kaggle.com/datasets/nltkdata/europarl). Use the english language for the task.


Evaluate your model based on perplexity and generate sentences using n-grams with n={2,3,4,5}. 

*Reading Material: https://web.stanford.edu/~jurafsky/slp3/3.pdf*

In [12]:
#Reading the data first which is extracted in a folder named english
folder_name = 'english'
corpus = []
for fname in os.listdir(folder_name):
    fname = os.path.join(folder_name, fname)
    with open(fname,'r',encoding="utf8") as f:
        lines = f.readlines()
        for line in lines:
            text = line.lower()
            filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
            translate_dict = dict((c, " ") for c in filters)
            translate_map = str.maketrans(translate_dict)
            text = text.translate(translate_map)
            sent = ' '.join(text.strip().split())
            if len(sent)>1:
                cleaned_sent = '<s> '+sent+' </s>'
                corpus.append(cleaned_sent)
                
#doing a train test split                
train_data, test_data = train_test_split(np.array(corpus),test_size=0.2,random_state=42)

In [13]:
### Your code here
class Ngram:
    def __init__ (self, n, alpha=1):
        self.n = n 
        self.alpha = alpha
        self.initials = ''
    
    def make_ngram(self, corpus):
        self.word_dict = {} #to store key value pairs
        self.key_counts = {} #to track count of keys
        self.togther_counts = {} #to track how many times they appeared togather
        for sent in corpus: #iterate over each sentence
            words = sent.split() 
            l = 0 #left pointer
            r = self.n-1 #right pointer
            
            while r<len(words): #iterate over each word
                key = tuple(words[l:r])
                val = words[r]
                togather = tuple(words[l:r] + [words[r]])
                
                if not key in self.word_dict:
                    self.word_dict[key] = {val:0}
                    self.key_counts[key] = 0
                    
                if not val in self.word_dict[key]:
                    self.word_dict[key][val] = 0
                   
                if not togather in self.togther_counts:
                    self.togther_counts[togather] = 0
                
                self.word_dict[key][val]+=1
                self.key_counts[key] +=1
                self.togther_counts[togather] +=1
                l+=1
                r+=1
                
            if not tuple([val]) in self.key_counts:
                self.key_counts[tuple([val])] = 0
            self.key_counts[tuple([val])] += 1
            self.build_vocab()
            
    def build_vocab(self):
        self.vocab = set()
        for k in self.key_counts.keys():
            for i in k:
                self.vocab.add(i) #this makes the vocabulory
    
    def generate_next_word(self, initials, c=0, max_sentence_len=100):
        #iterate over each word and find its prob but it's wiser to first calculate probs with words that appeared togather
        key = tuple(initials)
        
        #see if we have seen these initials before in corpus
        if key in self.word_dict:
            words = self.word_dict[key]
              
        #if not then we will pick some random words and check their probabilities with our initials
        else:
            idx = np.random.randint(len(self.vocab))
            word = list(self.vocab)
            words = {word[idx]:0}

        P = {}
        #see how many times this word appear with our last n words
        for word,count in words.items():
            if key in self.key_counts:
                prob = (count + self.alpha)/(self.key_counts[key]+self.alpha*(len(self.vocab)-1))
                P[word] = prob
            else:
                P[word] = (count + self.alpha)/(0+self.alpha*(len(self.vocab)-1))
        
        if len(self.initials) >max_sentence_len:
            return self.initials
        
        #pick the most probable word and predict it
        words_probabilities = sorted(P.items(), key=lambda x:x[1])
        #most probable word
        w, _ = words_probabilities[-1]

        if c==0:
            self.initials=initials

        self.initials += [w]

        if w == "</s>":
            return self.initials

        self.generate_next_word(self.initials[-self.n+1:],c+1)
            
    def measure_perplexity(self, sentences):
        P = [] #empty list to store probabilites
        N=0 #number of words
        for sent in sentences: #iterate over each sentence
            
            l = 0 #left pointer
            r = self.n-1 #right pointer
            
            #iterate till right pointer reaches the end of sentence
            while r<len(sent):
                N+=1 #number of words
                key = tuple(sent[l:r]) #the probability will be conditioned on the previous n words
                wi = sent[r] #the words whose probability we have to find

                #see if we have seen these initials before in corpus
                if key in self.word_dict:
                    #if the previous n words have occured togather get their count
                    w_ = self.key_counts[key]
                    
                    #if the word whose probability we have to find is in corpus, get its  count, else set the count to zero
                    if wi in self.word_dict[key]:
                        count = self.word_dict[key][wi]
                    else:
                        count = 0

                #if neither the initials nor the word has occured togather, we will set both to zero (laplace smoothing will take care)
                else:
                    count = 0
                    w_ = 0

                #to get the conditional probability of the word
                p=(count + self.alpha)/(w_+self.alpha*(len(self.vocab)-1))
                
                #to avoid getting smaller values take log
                P.append(np.log2(p))
                l+=1
                r+=1
        
        #transform it back and return 
        return 2**(-1/N*np.sum(P))

Let's first generate some text by our n-gram model with n=2,3,4,5. For keeping the scale of computations limited we will only train on first 1000 sentences and measure perplexity on them. This will give us good relative measure of the perplexity.

In [14]:
#function which generate sentences
def generate_sentences(text,n,train_data=train_data,train_size=1000):
    ng = Ngram(n,alpha=1)
    ng.make_ngram(train_data[:train_size])
    initials = text.split()[:n-1]
    ng.generate_next_word(initials)
    return ' '.join(ng.initials)

In [26]:
#Let's take a sample text as "european president has died" as a sample text and see what type sentences does our model generate
sample_text = 'government has announced elections'

#for n=2
print(generate_sentences(sample_text,n=2))

government </s>


In [27]:
#for n=3
generate_sentences(sample_text,n=3)

'government has expressed its satisfaction with the european union </s>'

In [28]:
#for n=4
generate_sentences(sample_text,n=4)

'government has announced november staff desirable confine günter elements attained stressing individuality accommodate vertical institutional clare inaugural securities draft christmas hopes danger tricked disadvantages over varying architect turns constitutes shape mighty applying appreciates striving personally concentrate reconstruction legal pardon match sea guarantee explained he grave started renovate absence speed thing timetable join doubtless resulted birds trouble dragged about clout tankers competitivity demonstrated swimming those growth honour exercised observations arrangement circumstances quote possibility effects peacefully intend injustices sailing reigns times sure managing card padania responsible decentralisation accused doubt employers memories lines labour memory solutions cases derivatives birds china example sections rules radical'

In [29]:
#for n=5
generate_sentences(sample_text,n=5)

'government has announced elections easier light forthcoming decentralising cooperate voted subjects rapporteur paragraph anyone protocols examine nothing dispose workings liabilities changing stressing management embargo therefore valid discussed 18 prospects dossiers hears being concretely intends margin inherent ahead fire gain shape suffering 35 reservoir whatsoever safety prevention potential fight forgive knörr debating year matters recreational million hopefully gnp regions opened flexible satisfactory dredging reserves compromise 10 dgs widely management quite liability conscience divided denouncing farming substances reliable limitation copyright discussed separate preparatory past foul alarming cent 69 precedence alongside huge simplify extent corruption september toned approved mothers exemption break arrested rights stand'

In [30]:
#to check perplexity
#to keep the size of the computations in control we will calculate perplexity over first 1000 sentences of training set
train = train_data[:1000]
test = test_data[:1000]

#perplexity of n=2 model
ng = Ngram(n=2,alpha=1)
ng.make_ngram(train)
print('The Perplexity of n=2 model is', ng.measure_perplexity(test))

The Perplexity of n=2 model is 3931.2996248844865


In [31]:
#perplexity of n=2 model
ng = Ngram(n=3,alpha=1)
ng.make_ngram(train)
print('The Perplexity of n=3 model is', ng.measure_perplexity(test))

The Perplexity of n=3 model is 3899.0071524044215


In [32]:
#perplexity of n=2 model
ng = Ngram(n=4,alpha=1)
ng.make_ngram(train)
print('The Perplexity of n=4 model is', ng.measure_perplexity(test))

The Perplexity of n=4 model is 3899.000000000003


In [33]:
#perplexity of n=2 model
ng = Ngram(n=5,alpha=1)
ng.make_ngram(train)
print('The Perplexity of n=5 model is', ng.measure_perplexity(test))

The Perplexity of n=5 model is 3897.999999999988


Note: We can observe the decreasing perplexity here, note that the perplexity is calculated on portion of train and test data, also the data was randomly sampled into training and test set. So a large value of perplexity does not neccessarily mean that it is wrong.